# Будем обучать Bert из базового библиотеки Replay

### Модель будет обучаться на решение задачи прогноза следующего фильма каждого пользователя.
### В качестве основы был взят пример из гита RePlay

https://github.com/sb-ai-lab/RePlay/blob/main/examples/10_bert4rec_example.ipynb

## 0. Установка и импорты нужных пакетов, библиотек и данных

Устанавливаем нужные пакеты

In [1]:
!pip install transformers
!pip install torch
!pip install tqdm
!pip install replay-rec
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.8/196.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 22.2 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
  Created wheel for fixed-install-nmslib: filename=fixed_install_nmslib-2.1.2-cp310-cp310-linux_x86_64.whl size=14023634 sha256=c3bfa5578bbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 46.7 MB/s eta 0:00:00


Импортируем библиотеки и фиксируем `seed`

In [2]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback,
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback,
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import Bert4Rec
from replay.models.nn.sequential.bert4rec import (
    Bert4RecPredictionDataset,
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionBatch,
    Bert4RecModel
)

import pandas as pd

import random
import numpy as np

def set_global_seed(seed: int) -> None:

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_global_seed(42)

import warnings
warnings.filterwarnings("ignore")

In [3]:
spark_session = get_spark_session()

In [4]:
data = pd.read_csv('events.csv')
users = pd.read_csv('user_features.csv')
items = pd.read_csv('item_features.csv')

## 1. Предобработка данных

Переобозначаем `timestamp`, не нарушая порядок: пользователь с одним и тем же timestamp-ом, вероятно, баг - поэтому для поддержания последовательности для модели необходимо переобозначение

In [5]:
data["timestamp"] = data["timestamp"].astype("int64")
data = data.sort_values(by="timestamp")
data["timestamp"] = data.groupby("user_id").cumcount()

После нескольких прогонов моделей с разными гиперпараметрами на основе валидационной выборки был выбран лучший набор, представленный в ячейках скрипта.</br></br>
Итоговую модель обучаем без val выборки, оставляем на контроле переобучения только тестовый датасет.

In [6]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(data)
raw_train_events, raw_train_gt = splitter.split(raw_test_events)

Функция для объединения в одну рабочую структуру данных информации об интерациях, пользователях и фильмах. Накидываем ее на тренировочную и тестовую выборки

In [7]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

In [8]:
train_dataset = Dataset(feature_schema=prepare_feature_schema(is_ground_truth=False),
                        interactions=raw_train_events,
                        query_features=users,
                        item_features=items,
                        check_consistency=True,
                        categorical_encoded=False)
test_dataset = Dataset(feature_schema=prepare_feature_schema(is_ground_truth=False),
                       interactions=raw_test_events,
                       query_features=users,
                       item_features=items,
                       check_consistency=True,
                       categorical_encoded=False)
test_gt = Dataset(feature_schema=prepare_feature_schema(is_ground_truth=True),
                  interactions=raw_test_gt,
                  check_consistency=True,
                  categorical_encoded=False)

Прописываем схему для соотнесения наших выборок с тензорным представлением для работы модели

In [28]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(name=ITEM_FEATURE_NAME,
                      is_seq=True,
                      feature_type=FeatureType.CATEGORICAL,
                      feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
                      feature_hint=FeatureHint.ITEM_ID,
                      embedding_dim=320) #тут пробовал варианты от 100 до 500
                            )

Используем `SequenceTokenizer` для токенизации данных в наших датасетах.</br>
- Токенизация учитывает последовательность интераций пользователя для построения токенов
- Учитывается внутренняя информация пользователей и айтемов
- Классическая BERT-овская токенизация с доп токенами и субтокенами `CLS`, `SEP` для последовательностей, а также `PAD`, `0` и `1` для `attention_mask`

In [29]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(test_dataset)
sequential_validation_gt = tokenizer.transform(test_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt)

test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

## 2. Создание объектов модели, трейнера и даталоадеров

В качестве основы модели выступает архитектура `BERT`:</br>
- Внутренний слой размера `hidden_size=300`
- Количество голов трансформера `head_count=4`
- Количество трансформерных блоков `block_count=2`
- Дропаут `dropout_rate=0.52`</br>
  При начальном обучении лучше всего себя показал вариант с `dropout_rate=0.5`. Для итогового предсказания, так как модель обучается уже на train+val, дропаут немного увеличим для контроля переобучения
- В качестве оптимизатора используем `FatOptimizerFactory` (под капотом `Adam`)

In [31]:
MAX_SEQ_LEN = 100
BATCH_SIZE = 256
NUM_WORKERS = 4

model = Bert4Rec(tensor_schema,
                 block_count=2,
                 head_count=8, # 4, 6, 8
                 max_seq_len=MAX_SEQ_LEN,
                 hidden_size=320, #100, 300, 500
                 dropout_rate=0.35, #0.3, 0.5, 0.7
                 optimizer_factory=FatOptimizerFactory(learning_rate=1e-3)) #1e-3, 1e-4

В качестве основной метрики для коллбеков используем `recall@10` как в итоговой метрике соревнования

In [32]:
checkpoint_callback = ModelCheckpoint(dirpath=".checkpoints",
                                      save_top_k=1,
                                      verbose=True,
                                      monitor="recall@10",
                                      mode="max")

validation_metrics_callback = ValidationMetricsCallback(metrics=["recall"],
                                                        ks=[10],
                                                        item_count=train_dataset.item_count,
                                                        postprocessors=[RemoveSeenItems(sequential_validation_dataset)])

csv_logger = CSVLogger(save_dir=".logs/train", name="RecommenderBERTModelv9")

trainer = L.Trainer(max_epochs=200,
                    callbacks=[checkpoint_callback, validation_metrics_callback],
                    logger=csv_logger)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [33]:
train_dataloader = DataLoader(
    dataset=Bert4RecTrainingDataset(sequential_train_dataset,
                                    max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True)

validation_dataloader = DataLoader(
    dataset=Bert4RecValidationDataset(sequential_validation_dataset,
                                      sequential_validation_gt,
                                      sequential_train_dataset,
                                      max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True)

## 3. Обучение модели

In [34]:
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | _model | Bert4RecModel    | 4.9 M  | train
1 | _loss  | CrossEntropyLoss | 0      | train
----------------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.495    Total estimated model params size (MB)
38        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | _model | Bert4RecModel    | 4.9 M  | train
1 | _loss  | CrossEntropyLoss | 0      | train
----------------------------------------------------
4.9 M     Trainable params
0         Non-trainable params
4.9 M     Total params
19.495    Total estimated model params size (M

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 24: 'recall@10' reached 0.03311 (best 0.03311), saving model to '/content/.checkpoints/epoch=0-step=24.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 24: 'recall@10' reached 0.03311 (best 0.03311), saving model to '/content/.checkpoints/epoch=0-step=24.ckpt' as top 1


...

INFO: Epoch 198, global step 4776: 'recall@10' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 198, global step 4776: 'recall@10' was not in top 1


k             10
recall  0.252152



Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 199, global step 4800: 'recall@10' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 199, global step 4800: 'recall@10' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=200` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=200` reached.


k             10
recall  0.253311



In [35]:
best_model = Bert4Rec.load_from_checkpoint(checkpoint_callback.best_model_path)

In [36]:
checkpoint_callback.best_model_path

'/content/.checkpoints/epoch=149-step=3600.ckpt'

In [37]:
torch.save(best_model.state_dict(), "./RecommenderBERTModelv9.pth")

## 4. Инференс модели

In [38]:
prediction_dataloader = DataLoader(
    dataset=Bert4RecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True)

csv_logger = CSVLogger(save_dir=".logs/test", name="RecommenderBERTModelv9")

В качестве коллбека используем специальный для `PySpark DataFrame`

In [39]:
TOPK = [10]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(spark_session=spark_session,
                                                    top_k=max(TOPK),
                                                    query_column="user_id",
                                                    item_column="item_id",
                                                    rating_column="score",
                                                    postprocessors=postprocessors)


Прокидываем в инференс тестовую выборку (последнее наблюдение датафрейма)

In [40]:
trainer = L.Trainer(callbacks=[spark_prediction_callback],
                    logger=csv_logger,
                    inference_mode=True)

trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [41]:
recs = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [42]:
result = recs.toPandas()
result

,score,user_id,item_id
0,10.241002,31,3298
1,6.744629,62,183
2,9.643677,303,512
3,6.908848,423,2161
4,7.031539,260,3597
...,...,...,...
60395,9.128984,5935,3035
60396,9.552566,5677,1527
60397,7.543508,5968,1527
60398,7.985178,6001,1848


In [30]:
# result.to_csv('bert_scores.csv', index=False, header=True)

## 5. Формируем итоговый submission

In [43]:
top_recs = (
    result.sort_values(['user_id', 'score'], ascending=[True, False])
    .groupby('user_id')
    .agg(top_items=('item_id', lambda x: ' '.join(map(str, x.head(10)))))
    .reset_index()
)

top_recs.columns = ['user_id', 'item_id']
top_recs['user_id'] = top_recs['user_id'].astype(int)
top_recs = top_recs.sort_values('user_id', ascending=True)
top_recs.to_csv('submit_The_boysV9.csv', index=False, header=True)

top_recs

,user_id,item_id
0,0,1422 2275 1461 3699 980 1052 1951 1014 2555 3214
1,1,232 3101 3656 1246 1686 1822 452 1459 1884 2467
2,2,2311 382 2774 234 1781 2424 221 724 2354 2428
3,3,1560 409 989 1223 487 124 707 1315 3272 3013
4,4,394 983 1191 3667 1044 1160 3598 755 2237 3602
...,...,...
6035,6035,1011 3382 3668 1376 2664 2256 124 3665 2366 1296
6036,6036,3692 401 3418 1011 2603 3382 2732 2470 2745 1811
6037,6037,2256 2664 1747 1375 1102 3059 3656 2646 2776 3418
6038,6038,450 3278 1680 154 1247 3605 998 2566 3073 1350
